# Synthethic generation of light curves

This notebooks illustrates the synthethic generation of light curves with mutis.

There are three methods implemented with mutis, with some variations:
- Generating signals by **sampling**. This signals have same statisical distribution.
- Gnerating signals by randomization of the Fourier transform from the **PSD** (Power Spectral Distribution). This signals have the same variability, and same mean and std.
- Generating signals by integration of an **stochastic** process (Orstein-Uhlenbeck process). Signals have similar shape.

In [ ]:
# I'm doing a lot of changes to MUTIS while writting this, better reload automatically.
%load_ext autoreload
%autoreload

import numpy as np
import pandas as pd

import matplotlib as mplt
import matplotlib.pyplot as plt

from astropy.time import Time

import  mutis

## Load test data

In [ ]:
data = {}

In [ ]:
data['3mm'] = pd.read_csv('data/mm-I.dat', comment='!')
data['3mm']

In [ ]:
data['gamma'] = pd.read_csv('data/gamma-I.dat', comment='!')
data['gamma']

## 3mm

In [ ]:
import mutis
sig3mm = mutis.Signal(data['3mm']['jyear'], data['3mm']['I'], data['3mm']['dI'])
sig3mm.plot()

### `lc_gen_samp`

We first check the generation with simple sampling of the signals. We see that the shape is not similar.

In [ ]:
sig3mm.check_gen('lc_gen_samp')

### `lc_gen_psd_*`

In [ ]:
sig3mm.check_gen('lc_gen_psd_fft')

We see that the signals have similar shape, and almost identical variability. The statistical distribution is not exactly the same.

We now check using the non-uniform fourier transform. The generation with the FFT was not technically correct since the signal was not evenly sampled in time.

In [ ]:
sig3mm.check_gen('lc_gen_psd_nft')

Still another method can be used to reconstruct the signal; the Lomb-Scargle method to compute the PSD.

In [ ]:
sig3mm.check_gen('lc_gen_psd_lombscargle')

**From these results, we see that the shape for 3mm is better reproduced with `lc_gen_psd_fft`.**

### `lc_gen_OU`

To use the stochastic OU method, first we need to find suitable parameters:

In [ ]:
sig3mm.OU_fit()

In [ ]:
sig3mm.check_gen('lc_gen_ou', fgen_params={'mu':1.8, 'sigma':1.01, 'theta':1.96})

In [ ]:
sig3mm.OU_mu, sig3mm.OU_sigma, sig3mm.OU_theta = 1.8, 1.01, 1.96

We see that these value produce synthethic light curves that have similar shape, distribution and variability.

## Gamma

In [ ]:
sigGamma = mutis.Signal(data['gamma']['jyear'][np.isfinite(data['gamma']['CFlux'])], 1e6*data['gamma']['CFlux'][np.isfinite(data['gamma']['CFlux'])], 1e6*data['gamma']['CFluxErr'][np.isfinite(data['gamma']['CFlux'])])
sigGamma.plot()

### `lc_gen_psd_*`

In [ ]:
sigGamma.check_gen('lc_gen_psd_fft')

Here we see that `lc_gen_psd_fft` does not generate signals with similar shapes, specially during flares.

In [ ]:
sigGamma.check_gen('lc_gen_psd_nft')

### `lc_gen_OU`

In [ ]:
sigGamma.OU_fit()

In [ ]:
sigGamma.check_gen('lc_gen_ou',
                   fgen_params={'mu':0.21, 'sigma':4.9, 'theta':7.6, 
                               })#'scale':np.std(sigGamma.values), 'loc':np.mean(sigGamma.values)})

## Set and generate synthethics

Now that we have checked which generation methods are the best, we set them and generate the synthethic light curves.

In [ ]:
%%time
sig3mm.fgen = 'lc_gen_psd_fft'
sig3mm.gen_synth(400)

In [ ]:
%%time
sigGamma.fgen = 'lc_gen_ou'
sigGamma.OU_mu, sigGamma.OU_sigma, sigGamma.OU_theta = 0.2, 4.9, 7
sigGamma.gen_synth(400)

## Correlation

In [ ]:
corr3mmGamma = mutis.Correlation(sig3mm, sigGamma, 'welsh')
corr3mmGamma.plot_signals()

In [ ]:
corr3mmGamma.gen_times(ftimes='uniform', tmin=-200/365, tmax=+200/365, n=50, nbinsmin=11)
corr3mmGamma.plot_times()
plt.xlim([-200/365,+200/365])

In [ ]:
corr3mmGamma.samples = 400

In [ ]:
%%time
corr3mmGamma.gen_corr(uncert=False)

In [ ]:
corr3mmGamma.plot_corr(uncert=False)
plt.xlim([-200/365,+200/365])